In [1]:
# imports
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

import numpy as np
import random
import sympy as sym
import winsound
from scipy.stats import beta
from sklearn.metrics import accuracy_score
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import winsound

In [2]:
duration = 1000  # milliseconds
freq = 440  # Hz

In [3]:
# function to count the number of the infected neighbores of i at t:\n",
def CNbr(G,X,n,T):
    C=np.zeros((T,n))
    for t in range(T):
        C[t]=G[t].dot(X.T[t])
    return C.T

In [4]:
def transition(X,t,G,j,param):
    alpha_=param[0]
    beta_=param[1]
    gama_=param[2]
    c=CNbr(G,X,n,T)[j][t]
    k=X[j,t]-2*X[j,t+1]
    if k==0:
        return 1-alpha_-beta_*c
    elif k==-2:
        return alpha_+beta_*c
    elif k==1:
        return gama_
    else:
        return 1-gama_

In [5]:
# Function to obtain the very initial sample of X, using forwad sampling:\n",
def Forward_Sampling(T,n,G,param):
    alpha_=param[0]
    beta_=param[1]
    gama_=param[2]
    P=1
    p0=P
    p1=1-P
    x=int(np.round(((1-P)*n),0))
    X=np.zeros((n,T))  
    idx=np.random.choice(range(n), x)
    X[idx,0]=1
    for t in range(T-1):
        for i in range(n):
            c=CNbr(G,X,n,T)[i,t]
            if X[i,t]==0:
                p1=(alpha_+beta_*c)
            else:
                p1=(1-gama_)
            X[i,t+1]=np.random.binomial( 1, p1,size=None) 
    return X 

In [6]:
# function to sample X, using Gibbs Sampling:\n",
def Sample_hidden_state(X,G,Y,param,t):
    pos_probs=np.zeros((n,T))
    alpha_=param[0]
    beta_=param[1]
    gama_=param[2]
    theta_0_0=param[3]
    theta_1_0=param[4] 
    theta_0_1=param[5]
    theta_1_1=param[6] 
    theta_0_2=param[7]
    theta_1_2=param[8]
    theta_0_3=param[9]
    theta_1_3=param[10] 
    theta_0_4=param[11]
    theta_1_4=param[12] 
    theta_0_5=param[13]
    theta_1_5=param[14] 
    for i in range(n):
        if t==0:
            p0,p1=P,1-P
        else:
            p0,p1=1,1
        pow00=int(Y[0,i,t] == 0)   
        pow10=int(Y[0,i,t] == 1)
        pow01=int(Y[1,i,t] == 0)  
        pow11=int(Y[1,i,t] == 1)    
        pow02=int(Y[2,i,t] == 0)      
        pow12=int(Y[2,i,t] == 1)
        pow03=int(Y[3,i,t] == 0)  
        pow13=int(Y[3,i,t] == 1)
        pow04=int(Y[4,i,t] == 0)  
        pow14=int(Y[4,i,t] == 1)    
        pow05=int(Y[5,i,t] == 0)      
        pow15=int(Y[5,i,t] == 1)     
        p0=p0*(1-theta_0_0)**pow00
        p1=p1*(1-theta_1_0)**pow00
        p0=p0*(1-theta_0_1)**pow01
        p1=p1*(1-theta_1_1)**pow01
        p0=p0*(1-theta_0_2)**pow02
        p1=p1*(1-theta_1_2)**pow02
        p0=p0*(1-theta_0_3)**pow03
        p1=p1*(1-theta_1_3)**pow03
        p0=p0*(1-theta_0_4)**pow04
        p1=p1*(1-theta_1_4)**pow04
        p0=p0*(1-theta_0_5)**pow05
        p1=p1*(1-theta_1_5)**pow05
        p0=p0*(theta_0_0)**pow10
        p1=p1*theta_1_0**pow10
        p0=p0*(theta_0_1)**pow11
        p1=p1*theta_1_1**pow11
        p0=p0*(theta_0_2)**pow12
        p1=p1*theta_1_2**pow12
        p0=p0*(theta_0_3)**pow13
        p1=p1*theta_1_3**pow13
        p0=p0*(theta_0_4)**pow14
        p1=p1*theta_1_4**pow14
        p0=p0*(theta_0_5)**pow15
        p1=p1*theta_1_5**pow15
              
       # emission probability when sample is X[i,t]=0:
    
        X[i,t]=0
        p_0=p0
        if (t==0):
            c=CNbr(G,X,n,T)[i][t]
        else:  
            c=CNbr(G,X,n,T)[i][t-1]
              
       # transition probability of i from t-1 to t, when sample is X[i,t]=0:
       
        if t!=0:
            if X[i,t-1]==0:
                p_0=p_0*(1-alpha_-beta_*c)
            else:
                p_0=p_0*gama_
    
       # emission probability when sample is X[i,t]=1:
        X[i,t]=1
        p_1=p1
        if (t==0):
            c=CNbr(G,X,n,T)[i][t]
        else:    
            c=CNbr(G,X,n,T)[i][t-1]
        if t!=0:
        # transition probability of i, when sample is X[i,t]=1:
            if X[i,t-1]==0:
                p_1=p_1*(alpha_+beta_*c)
            else:
                p_1=p_1*(1-gama_)
        # transition probability of i's neighbores, from t to t+1:
        if t!=T-1:        
            X[i,t]=0
            for j in np.where(G[t][i]==1)[0]:
                p_0=p_0*transition(X,t,G,j,param)

            X[i,t]=1
            for j in np.where(G[t][i]==1)[0]:
                p_1=p_1*transition(X,t,G,j,param)
        if p_0+p_1==0:            
            l=0.5
        else:
            l=p_1/(p_0+p_1)
        X[i,t]=np.random.binomial( 1,  l,size=None) 
        pos_probs[i,t]=l
    return X ,pos_probs   

In [7]:
# Gibbs sampling to obtain X, as new sample of posterior distribution:
def Calculate_X(K,T,X,G,Y,param):
    for k in range(K):
        for t in range(T):
            hidden_states=Sample_hidden_state(X,G,Y,param,t)
            X=hidden_states[0]
            pos_probs=hidden_states[1]
    return X , pos_probs

In [8]:
def Accuracy_(X,X_):
    Xflat = np.hstack(np.hstack(X))
    X_flat = np.hstack(np.hstack(X_))
    result=accuracy_score(Xflat, X_flat,normalize=True)
    return result

In [9]:
# function to define auxiliary variable R_(n,t):
def R_(G,X,params):
    alpha_,beta_,gama_=params[0],params[1],params[2]
    
    R=np.zeros((n,T))+1
    for i in range(n):
        for t in range(T-1):
            infected_neighbore=int(CNbr(G,X,n,T)[i,t])
            pr_a=alpha_/(alpha_+beta_*infected_neighbore)
            pr_b=beta_/(alpha_+beta_*infected_neighbore)
            v=np.random.multinomial(1, [pr_a]+[pr_b]*infected_neighbore)
            if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

In [10]:
# function to sample new parameters and update parameters:
def Params(R,G,X,n,T,Y,param):
        
    alpha_=param[0]
    beta_=param[1]
    gama_=param[2]
    #theta_0_0=param[3]
    #theta_1_0=param[4] 
    #theta_0_1=param[5]
    #theta_1_1=param[6] 
    #theta_0_2=param[7]
    #theta_1_2=param[8]
    #theta_0_3=param[9]
    #theta_1_3=param[10] 
    #theta_0_4=param[11]
    #theta_1_4=param[12] 
    #theta_0_5=param[13]
    #TP0=np.sum(np.multiply(X,Y[0]))
    #FP0=np.count_nonzero(X-Y[0]==-1)
    #TP1=np.sum(np.multiply(X,Y[1]))
    #FP1=np.count_nonzero(X-Y[1]==-1)
    #TP2=np.sum(np.multiply(X,Y[2]))
    #FP2=np.count_nonzero(X-Y[2]==-1)
    #TP3=np.sum(np.multiply(X,Y[3]))
    #FP3=np.count_nonzero(X-Y[3]==-1)
    #TP4=np.sum(np.multiply(X,Y[4]))
    #FP4=np.count_nonzero(X-Y[4]==-1)
    #TP5=np.sum(np.multiply(X,Y[5]))
    #FP5=np.count_nonzero(X-Y[5]==-1)    
    infR=np.array(CNbr(G,X,n,T))
    alpha_=Sample_alpha(a_alpha +  np.count_nonzero(R==0) , b_alpha +np.count_nonzero(X==0)- np.count_nonzero(R==0))
    beta_=Sample_beta(a_beta + np.count_nonzero(R==2) , b_beta +np.sum(np.multiply((1-X),infR))-np.count_nonzero(R==2))
    gama_=Sample_gama(a_gama +(T-1)*n-np.count_nonzero(X[:,:-1]-X[:,1:]-1), b_gama+np.sum(X)-(T-1)*n+np.count_nonzero(X[:,:-1]-X[:,1:]-1))
    theta_0_0=.01#Sample_theta00( a_teta00+FP0,b_teta00+np.count_nonzero(X==0)-FP0)
    theta_1_0=.99#Sample_theta10( a_teta10+TP0,b_teta10+np.count_nonzero(X==1)-TP0)
    theta_0_1=.01#Sample_theta01( a_teta01+FP1,b_teta01+np.count_nonzero(X==0)-FP1)
    theta_1_1=.99#Sample_theta11( a_teta11+TP1,b_teta11+np.count_nonzero(X==1)-TP1)
    theta_0_2=.01#Sample_theta02( a_teta02+FP2,b_teta02+np.count_nonzero(X==0)-FP2)
    theta_1_2=.99#Sample_theta12( a_teta12+TP2,b_teta12+np.count_nonzero(X==1)-TP2)
    theta_0_3=.01#Sample_theta03( a_teta03+FP3,b_teta03+np.count_nonzero(X==0)-FP3)
    theta_1_3=.99#Sample_theta13( a_teta13+TP3,b_teta13+np.count_nonzero(X==1)-TP3)
    theta_0_4=.01#Sample_theta04( a_teta04+FP4,b_teta04+np.count_nonzero(X==0)-FP4)
    theta_1_4=.99#Sample_theta14( a_teta14+TP4,b_teta14+np.count_nonzero(X==1)-TP4)
    theta_0_5=.01#Sample_theta05( a_teta05+FP5,b_teta05+np.count_nonzero(X==0)-FP5)
    theta_1_5=.99#Sample_theta15( a_teta15+TP5,b_teta15+np.count_nonzero(X==1)-TP5)

    R=R_(G,X,params)
    param=np.array([alpha_,beta_,gama_,theta_0_0,theta_1_0,theta_0_1,theta_1_1,theta_0_2,theta_1_2,theta_0_3,theta_1_3,theta_0_4,theta_1_4,theta_0_5,theta_1_5])
    return param,R

# function to sample from beta distribution

In [11]:
def Sample_alpha(a_alpha, b_alpha):
    for i in beta.rvs(a_alpha, b_alpha, size=10000):
        print("al",i,a_alpha,b_alpha)
        if (i>0.001)&(i<0.01):
            alpha_=round(i,3)
            break
    return alpha_    

In [12]:
def Sample_beta(a_beta, b_beta):
    for i in beta.rvs(a_beta, b_beta, size=10000):
        print("be",a_beta, b_beta)
        if (i>0.0001)&(i<0.051):
            beta_=round(i,4)
            break
    return beta_ 

In [13]:
def Sample_gama(a_gama,b_gama):
    for i in beta.rvs(a_gama, b_gama, size=10000):
        print("g",i,a_gama,b_gama)
        if (i>0.02)&(i<0.7):
            gama_=round(i,3)
            break
    return gama_  

In [14]:
def Sample_theta00(a_teta00, b_teta00):
    for i in beta.rvs(a_teta00, b_teta00, size=10000):
        #print("t00",a_teta00,b_teta00)
        if (i>0.01)&(i<0.3):
            theta_0_0=round(i,3)    
            break
    return theta_0_0 

In [15]:
def Sample_theta10(a_teta10, b_teta10):
    for i in beta.rvs(a_teta10, b_teta10, size=10000):
        #print("t10",i,a_teta10, b_teta10)
        if .993>round(i,3)>0.5:
            theta_1_0=round(i,3)
            break
    return theta_1_0 

In [16]:
def Sample_theta01(a_teta01, b_teta01):
    for i in beta.rvs(a_teta01, b_teta01, size=10000):
        #print("t01",i,a_teta01, b_teta01)
        if (i>0.01)&(i<0.3):
            theta_0_1=round(i,3)
            break
    return theta_0_1

In [17]:
def Sample_theta11(a_teta11, b_teta11):
    for i in beta.rvs(a_teta11, b_teta11, size=10000):
        #print("t11",i,a_teta11, b_teta11)
        if .993>round(i,3)>0.5:
            theta_1_1=round(i,3)
            break
    return theta_1_1

In [18]:
def Sample_theta02(a_teta02, b_teta02):
    for i in beta.rvs(a_teta02, b_teta02, size=10000):
        #print("t02",i,a_teta02, b_teta02)
        if (i>0.01)&(i<0.3):
            theta_0_2=round(i,3)
            break
    return theta_0_2 

In [19]:
def Sample_theta12(a_teta12, b_teta12):
    for i in beta.rvs(a_teta12, b_teta12, size=10000):
        #print("t12",i,a_teta12,b_teta12,i)
        if .993>round(i,3)>0.5:
            theta_1_2=round(i,3)
            break
    return theta_1_2

In [20]:
def Sample_theta03(a_teta03, b_teta03):
    for i in beta.rvs(a_teta03, b_teta03, size=10000):
        #print("t03",i,a_teta03, b_teta03)
        if (i>0.01)&(i<0.3):
            theta_0_3=round(i,3)
            break
    return theta_0_3  

In [21]:
def Sample_theta13(a_teta13, b_teta13):
    for i in beta.rvs(a_teta13, b_teta13, size=10000):
        #print("t13",i,a_teta13, b_teta13)
        if .993>round(i,3)>0.5:
            theta_1_3=round(i,3)
            break
    return theta_1_3

In [22]:
 def Sample_theta04(a_teta04, b_teta04):
    for i in beta.rvs(a_teta04, b_teta04, size=10000):
        #print("t04",i,a_teta04, b_teta04)
        if (i>0.01)&(i<0.3):
            theta_0_4=round(i,3)
            break
    return theta_0_4  

In [23]:
def Sample_theta14(a_teta14, b_teta14):
    for i in beta.rvs(a_teta14, b_teta14, size=10000):
        #print("t14",i,a_teta14, b_teta14)
        if .993>round(i,3)>0.5:
            theta_1_4=round(i,3)
            break
    return theta_1_4

In [24]:
def Sample_theta05(a_teta05, b_teta05):
    for i in beta.rvs(a_teta05, b_teta05, size=10000):
        #print("t05",i,a_teta05, b_teta05)
        if (i>0.01)&(i<0.3):
            theta_0_5=round(i,3)
            break
    return theta_0_5

In [25]:
def Sample_theta15(a_teta15, b_teta15):
    for i in beta.rvs(a_teta15, b_teta15, size=10000):
        #print("t15",a_teta15, b_teta15)
        if .993>round(i,3)>0.5:
            theta_1_5=round(i,3)
            break
    return theta_1_5

In [26]:
# initialize parameters for beta distributions:
a_alpha=1
b_alpha=1
a_beta=1
b_beta=5
a_gama=1
b_gama=5
    
a_teta00=1
b_teta00=5
a_teta10=800
b_teta10=1
a_teta01=1
b_teta01=5
a_teta11=800
b_teta11=1
a_teta02=1
b_teta02=5
a_teta12=800
b_teta12=1
a_teta03=1
b_teta03=5
a_teta13=800
b_teta13=1
a_teta04=1
b_teta04=5
a_teta14=800
b_teta14=1
a_teta05=1
b_teta05=5
a_teta15=800
b_teta15=1
# infection rate
P=1
hyper_params=[a_alpha,b_alpha,a_beta,b_beta,a_gama,b_gama,a_teta00,b_teta00,a_teta10,b_teta10,a_teta01,b_teta01,a_teta11,b_teta11,a_teta02,b_teta02,a_teta12,b_teta12,a_teta03,b_teta03,a_teta13,b_teta13,a_teta04,b_teta04,a_teta14,b_teta14,a_teta05,b_teta05,a_teta15,b_teta15]
   

In [27]:
# function to sample infection and emission parameters(alpha,beta,betaf,gama,teta0,teta1)\n",
def initialize_parameters(hyper_params):
    a_alpha=hyper_params[0]
    b_alpha=hyper_params[1]
    a_beta=hyper_params[2]
    b_beta=hyper_params[3]
    a_gama=hyper_params[4]
    b_gama=hyper_params[5]
    a_teta00=hyper_params[6]
    b_teta00=hyper_params[7]
    a_teta10=hyper_params[8]
    b_teta10=hyper_params[9]
    a_teta01=hyper_params[10]
    b_teta01=hyper_params[11]
    a_teta11=hyper_params[12]
    b_teta11=hyper_params[13]
    a_teta02=hyper_params[14]
    b_teta02=hyper_params[15]
    a_teta12=hyper_params[16]
    b_teta12=hyper_params[17]
    a_teta03=hyper_params[18]
    b_teta03=hyper_params[19]
    a_teta13=hyper_params[20]
    b_teta13=hyper_params[21]
    a_teta04=hyper_params[22]
    b_teta04=hyper_params[23]
    a_teta14=hyper_params[24]
    b_teta14=hyper_params[25]
    a_teta05=hyper_params[26]
    b_teta05=hyper_params[27]
    a_teta15=hyper_params[28]
    b_teta15=hyper_params[29]
    alpha_=Sample_alpha(a_alpha, b_alpha)
    beta_=Sample_beta(a_beta, b_beta)
    gama_=Sample_gama(a_gama,b_gama)
    theta_0_0=Sample_theta00(a_teta00, b_teta00)
    theta_1_0=Sample_theta10(a_teta10, b_teta10)
    theta_0_1=Sample_theta01(a_teta01, b_teta01)
    theta_1_1=Sample_theta11(a_teta11, b_teta11)
    theta_0_2=Sample_theta02(a_teta02, b_teta02)
    theta_1_2=Sample_theta12(a_teta12, b_teta12)
   
    theta_0_3=Sample_theta03(a_teta03, b_teta03)
    theta_1_3=Sample_theta13(a_teta13, b_teta13)
    
    theta_0_4=Sample_theta04(a_teta04, b_teta04)
    theta_1_4=Sample_theta14(a_teta14, b_teta14)
    theta_0_5=Sample_theta05(a_teta05, b_teta05)
    theta_1_5=Sample_theta15(a_teta15, b_teta15)
    params=np.array([alpha_,beta_,gama_,theta_0_0,theta_1_0,theta_0_1,theta_1_1,theta_0_2,theta_1_2,theta_0_3,theta_1_3,theta_0_4,theta_1_4,theta_0_5,theta_1_5])
    return params

In [28]:
# function to add noise to observation vector, in order to test the model performance:\n",
def add_noise(Y,noise_percent):
    Y_noisey=np.copy(Y)
    for turn in range(6):
        index=random.sample(range(0,n*T ), int(n*T*noise_percent))
        for j in index:
            feature=random.randint(0, 5)
            i=int(j/T)
            t=j-i*T
            Y_noisey[feature,i,t]=(1+Y[feature,i,t])*(1-Y[feature,i,t])
    return Y_noisey  

In [29]:
# load data sets for Y,G and generate first J samples of X:
%store -r Y
%store -r G
n,T=G.shape[1],G.shape[0]
U=20
K=40
J=20
params=initialize_parameters(hyper_params)
params#=np.array([0.006 , 0.0168, 0.305 , 0.045 , 0.992 , 0.096 , 0.992 , 0.185 ,
   #    0.992 , 0.192 , 0.991 , 0.251 , 0.99  , 0.289 , 0.992 ])

al 0.8481218624953761 1 1
al 0.6882102282243469 1 1
al 0.7306951293925686 1 1
al 0.22125286973541183 1 1
al 0.1498136007856174 1 1
al 0.5257466154179097 1 1
al 0.5766582606935631 1 1
al 0.3589490274513823 1 1
al 0.7468942940536681 1 1
al 0.18554788250494275 1 1
al 0.15018060839889166 1 1
al 0.45846629122611443 1 1
al 0.9065113737849972 1 1
al 0.9307347382041127 1 1
al 0.41158715454611633 1 1
al 0.761300959566073 1 1
al 0.13342304603078436 1 1
al 0.24844331092179095 1 1
al 0.14686128875263013 1 1
al 0.2824568296443716 1 1
al 0.7306482104788005 1 1
al 0.04165737120254416 1 1
al 0.0010854125288592231 1 1
be 1 5
be 1 5
be 1 5
be 1 5
g 0.24799729719149402 1 5


array([0.001, 0.031, 0.248, 0.014, 0.992, 0.029, 0.992, 0.224, 0.988,
       0.261, 0.992, 0.041, 0.991, 0.193, 0.992])

In [30]:
Y_noisey=add_noise(Y,.1)
Accuracy_(Y,Y_noisey)

0.9038800705467372

In [31]:
# Main code to run entire Gibbs sampling algorithm U times:
def Gibbs_train(hyper_params,T,n,G,Y,U,K,J):
    
    XX=[]
    prob=[]
    for j in range(J):
        params=initialize_parameters(hyper_params)
        print(params)
        X=Forward_Sampling(T,n,G,params)
        print("Forward_Sampling",j)
        print("Start Gibbs Sampling:")
        mylist=[]
        accuracy=[]
        for i in range(U):
            print("************************************************ Iteration",i,"************************************************")
            print("Current Parameters:",params)
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            print("Current Time is :", current_time)
            cal=Calculate_X(K,T,X,G,Y,params)
            X=cal[0]
            pos_probs=cal[1]
            R=R_(G,X,params)
            if i!=U-1:
                prm=Params(R,G,X,n,T,Y,hyper_params)
                params=prm[0]
                R=prm[1]
        
        XX.append(X) 
        prob.append(pos_probs)
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Current Time is :", current_time)
    winsound.Beep(freq, duration) 
    return np.array(XX) ,prob, params   

In [ ]:
Train=Gibbs_train(hyper_params,T,n,G,Y,U,K,J)

al 0.3712103904781696 1 1
al 0.3056174247246331 1 1
al 0.9364244558794332 1 1
al 0.2987698478371366 1 1
al 0.8868184929334777 1 1
al 0.1672129656923315 1 1
al 0.4498585168836345 1 1
al 0.35254197657083364 1 1
al 0.78985039395069 1 1
al 0.009203583975434452 1 1
be 1 5
be 1 5
be 1 5
be 1 5
be 1 5
be 1 5
be 1 5
be 1 5
g 0.022289317650452122 1 5
[0.009  0.0472 0.022  0.017  0.991  0.072  0.991  0.177  0.99   0.016
 0.992  0.264  0.992  0.1    0.99  ]
Forward_Sampling 0
Start Gibbs Sampling:
************************************************ Iteration 0 ************************************************
Current Parameters: [0.009  0.0472 0.022  0.017  0.991  0.072  0.991  0.177  0.99   0.016
 0.992  0.264  0.992  0.1    0.99  ]
Current Time is : 12:20:42
al 0.00015272269904402622 1 4150
al 0.00021180958161458205 1 4150
al 0.000761542264515925 1 4150
al 0.000535806094008157 1 4150
al 0.00010096877573750693 1 4150
al 1.273387794125065e-05 1 4150
al 5.099874422134707e-05 1 4150
al 0.00012323291141

************************************************ Iteration 6 ************************************************
Current Parameters: [0.002  0.0102 0.587  0.01   0.99   0.01   0.99   0.01   0.99   0.01
 0.99   0.01   0.99   0.01   0.99  ]
Current Time is : 12:58:55
al 0.0003188104089400264 1 4120
al 0.00014042362537664617 1 4120
al 0.000821883313872543 1 4120
al 3.1770434002319945e-05 1 4120
al 2.544667849801887e-05 1 4120
al 6.282732349825714e-05 1 4120
al 9.297490121581756e-05 1 4120
al 0.00044296578287412547 1 4120
al 0.00030494527008553764 1 4120
al 9.357170203115698e-05 1 4120
al 7.601474141507904e-05 1 4120
al 0.00029050536710177534 1 4120
al 0.00029362027654033905 1 4120
al 0.00038386815575277926 1 4120
al 1.045076822029175e-05 1 4120
al 0.0003644264634814477 1 4120
al 0.00010822898891035654 1 4120
al 0.000421705741962821 1 4120
al 5.191390125187923e-05 1 4120
al 6.841026820732536e-05 1 4120
al 0.0013323730528463289 1 4120
be 1 159.0
g 0.6960661286090392 25 20.0
*******************

************************************************ Iteration 11 ************************************************
Current Parameters: [0.001  0.0046 0.499  0.01   0.99   0.01   0.99   0.01   0.99   0.01
 0.99   0.01   0.99   0.01   0.99  ]
Current Time is : 13:30:28
al 0.00019249005335711747 1 4116
al 0.0006511184722018558 1 4116
al 5.581850398099439e-05 1 4116
al 0.00036617284199804226 1 4116
al 0.0001543243884349325 1 4116
al 0.0007211477958139322 1 4116
al 1.1301445390237027e-05 1 4116
al 0.00033154543886547183 1 4116
al 0.00016995720619172077 1 4116
al 0.00019048588450562052 1 4116
al 9.161962488380919e-05 1 4116
al 1.1102473517048522e-05 1 4116
al 0.00018909078290977382 1 4116
al 0.00045399202474605087 1 4116
al 0.00015243991810811 1 4116
al 0.0003813185487121693 1 4116
al 1.9172043992407853e-05 1 4116
al 9.061533979889109e-05 1 4116
al 0.000356716763219954 1 4116
al 6.9482399472825626e-06 1 4116
al 0.00035421341008342607 1 4116
al 5.720109033154449e-05 1 4116
al 0.000132654059221744

************************************************ Iteration 15 ************************************************
Current Parameters: [0.001  0.0203 0.56   0.01   0.99   0.01   0.99   0.01   0.99   0.01
 0.99   0.01   0.99   0.01   0.99  ]
Current Time is : 13:55:55
al 1.877076484569062e-05 1 4122
al 0.0006036848165990624 1 4122
al 2.4506426480117367e-05 1 4122
al 0.0002988409431445323 1 4122
al 2.4688197384999507e-05 1 4122
al 0.0002067797719675955 1 4122
al 8.841204145598797e-05 1 4122
al 0.0006200973184539861 1 4122
al 0.00024068169626547962 1 4122
al 0.00047213321612009655 1 4122
al 1.4192062192665498e-05 1 4122
al 0.0002855539120090665 1 4122
al 0.0002051871475161658 1 4122
al 0.0007467659058515824 1 4122
al 0.0004155250156640728 1 4122
al 3.85840124380425e-05 1 4122
al 0.00018979081426695457 1 4122
al 0.00017437563016395752 1 4122
al 0.000517893477831219 1 4122
al 0.00012947086026040522 1 4122
al 0.0004878595173620549 1 4122
al 0.00030558505703893763 1 4122
al 8.538714521475381e-05 

************************************************ Iteration 19 ************************************************
Current Parameters: [0.001  0.0061 0.506  0.01   0.99   0.01   0.99   0.01   0.99   0.01
 0.99   0.01   0.99   0.01   0.99  ]
Current Time is : 14:21:12
Current Time is : 14:27:33
al 0.654813874439806 1 1
al 0.9107001201981869 1 1
al 0.05178144103763941 1 1
al 0.3341732467378059 1 1
al 0.9774105016891761 1 1
al 0.9935206099544676 1 1
al 0.8227440045108941 1 1
al 0.4076524704366189 1 1
al 0.9911062426282273 1 1
al 0.4462126362590381 1 1
al 0.37202481064700654 1 1
al 0.6545255867267751 1 1
al 0.3470643690698813 1 1
al 0.5097100842046156 1 1
al 0.9041772682545811 1 1
al 0.25593516502137537 1 1
al 0.28001031322387654 1 1
al 0.7942061956882401 1 1
al 0.2910786648353906 1 1
al 0.17382991612467596 1 1
al 0.45349689738798 1 1
al 0.9486529360655652 1 1
al 0.5770928313912781 1 1
al 0.8522927680579552 1 1
al 0.908287263173098 1 1
al 0.4371677993363367 1 1
al 0.20870234263603887 1 1
al 0.

In [ ]:
def estimate_Y(X,param):
    theta_0_0=param[3]
    theta_1_0=param[4]
    theta_0_1=param[5]
    theta_1_1=param[6] 
    theta_0_2=param[7]
    theta_1_2=param[8]
    theta_0_3=param[9]
    theta_1_3=param[10] 
    theta_0_4=param[11]
    theta_1_4=param[12]
    theta_0_5=param[13]
    theta_1_5=param[14]
    theta0=[theta_0_0,theta_0_1,theta_0_2,theta_0_3,theta_0_4,theta_0_5]
    theta1=[theta_1_0,theta_1_1,theta_1_2,theta_1_3,theta_1_4,theta_1_5]
    Y_=np.zeros((6,n,T))
    for t in range(T):
        for i in range(n):
            for j in range(6):
                p1=theta0[j]**(1-X[i,t])*(theta1[j])**X[i,t]
                p0=(1-theta0[j])**(1-X[i,t])*(1-theta1[j])**X[i,t]
                l=p1/(p1+p0)
                Y_[j,i,t]=np.random.binomial( 1, l,size=None)
    return Y_

In [ ]:
X=Train[0]
pos_probs=Train[1]
params=Train[2]

In [ ]:
params

In [ ]:
Y_estimate=estimate_Y(X,params)

In [ ]:
def Gibbs_test(params,T,n,G,Y,K,J):
       
    acc=[]  
    XX=[]
    prob=[]
    for j in range(J):
        #params=initialize_parameters(hyper_params)
        print(params)
        X=Forward_Sampling(T,n,G,params)
        print("Forward_Sampling",j,"\n",X)
        print("Start Gibbs Sampling:")
        cal=Calculate_X(K,T,X,G,Y,params)
        X=cal[0]
        label="Sample:"+str(j)
        pos_probs=cal[1]
        prob.append(pos_probs)    
        XX.append(X)    
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Current Time is :", current_time)
    winsound.Beep(freq, duration) 
    return np.array(XX),prob
   

In [ ]:
Test=Gibbs_test(params,T,n,G,Y_estimate,K,J)

In [ ]:
pos_probs=Test[1][0]
y_score=np.hstack(pos_probs)
y_score.shape

In [ ]:
X_=Test[0][1]
y_test=np.hstack(X)
y_test.shape

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_score,pos_label=1)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
lw = 2
plt.plot(
    fpr,
    tpr,
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f)" % roc_auc,
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Synthetized data:
# Function to generates synthetic dataset
def Synthetic_Data(n,T,y,params):
    Y_=np.zeros((6,n,T))
    alpha_,beta_,gama_,theta_0_0,theta_1_0,theta_0_1,theta_1_1,theta_0_2,theta_1_2,theta_0_3,theta_1_3,theta_0_4,theta_1_4,theta_0_5,theta_1_5=params[0],params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8],params[9],params[10],params[11],params[12],params[13],params[14]
    teta_0=[theta_0_0,theta_0_1,theta_0_2,theta_0_3,theta_0_4,theta_0_5]
    teta_1=[theta_1_0,theta_1_1,theta_1_2,theta_1_3,theta_1_4,theta_1_5]
    P=.95
    x=int(np.round(((1-P)*n),0))
    X=np.zeros((n,T))
    idx=np.random.choice(range(n), x)
    X[idx,0]=1
    # Random social network
    G=[]
    for j in range(T):
        g=np.identity(n,dtype=int)
        for i in range(n):
            indx=np.random.choice(range(i,n), y)
            for inx in indx:
                g[i,inx]=1  
                g[inx,i]=1
        G.append(g)
    G=np.array(G)
  
    # Synthetize X, using params,G, Family and transition probability:
    for t in range(T-1):
        for i in range(n):
            c=CNbr(G,X,n,T)[i,t]
            if X[i,t]==0:
                p1=alpha_+beta_*c
            else:
                p1=1-gama_
            
            X[i,t+1]=np.random.binomial( 1, p1,size=None) 
            
    # Synthetize Y, using params, G, X, emission probability:
    for t in range(T):
        for i in range(n):
            for j in range(6):
                if X[i,t]==0:
                    teta0=teta_0[j]
                    Y_[j,i,t]=np.random.binomial( 1, teta0,size=None) 
                else:
                    teta1=teta_1[j]
                    Y_[j,i,t]=np.random.binomial( 1, teta1,size=None) 
    return G,Y_,X          

In [ ]:
params=np.array([.005,.045,.3,.01,.99,.01,.99,.01,.99,.01,.99,.01,.99,.01,.99])

In [ ]:
# Generate synthetic data,G ,Y:
n,T,y=10,10,2
synthetic_data=Synthetic_Data(n,T,y,params)
G,Y_true,X_true =synthetic_data[0],synthetic_data[1],synthetic_data[2]

In [ ]:
print(G)

In [ ]:
Train=Gibbs_train(hyper_params,T,n,G,Y_true,U,K,J)

In [ ]:
y_test=X_true

In [ ]:
Train[0][3]

In [ ]:
Test=Gibbs_test(params,T,n,G,Y_true,K,J)

In [ ]:
for j in range(J):
    pos_probs=Test[0][j]
    y_score=np.hstack(pos_probs)
    y_test=np.hstack(X_true)
    fpr, tpr, thresholds = roc_curve(y_test, y_score,pos_label=1)
    plt.figure()
    lw = 2
    plt.plot(
        fpr,
        tpr,
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % roc_auc,
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic example")
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Binarize the output
y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]

# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(
    svm.SVC(kernel="linear", probability=True, random_state=random_state)
)
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(
    fpr[2],
    tpr[2],
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f)" % roc_auc[2],
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()